In [5]:
# Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException


# Rest
from bs4 import BeautifulSoup
import time
import pandas as pd
import datetime
import h5py
import numpy as np

# Global settings for the driver
chrome_options = Options()
chrome_options.add_experimental_option(
    "detach", True
)  # keeps driver open until manual termination


In [2]:
def launch_chrome(url):
    """A method to install the latest Chrome driver and get url
    ---------
    Parameters:
    url = url defined by user"""

    global driver
    # installing driver making sure that new driver is up to date
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    return driver


In [11]:
top_crypto = [
    "bitcoin",
    "ethereum",
    "tether",
    "usd-coin",
    "bnb",
    "xrp",
    "binance-usd",
    "cardano",
    "solana",
    "dogecoin",
    "polkadot-new",
    "polygon",
    "tron",
    "HEX",
]

crypto_project_info = []

for crypto in top_crypto:
    try:
        launch_chrome(f"https://coinmarketcap.com/currencies/{crypto}/project-info/")
        driver.implicitly_wait(10)  # seconds

        driver.maximize_window()

        # Not all pages have an additional layer in dynamic component. If there is no dynamic component, the try block is not needed.
        try:
            # Cookie element blocks the selection of target element "Social Stats"
            cookie = driver.find_element(
                By.XPATH, '//*[@id="cmc-cookie-policy-banner"]/div[2]'
            )
            cookie.click()
            # Click on element "Social Stats" to access target data
            element = driver.find_element(
                By.XPATH,
                '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/div[4]',
            )
            WebDriverWait(driver, timeout=5)
            element.click()
            wait = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "selected"))
            )
        except NoSuchElementException:
            pass

        github_commits = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]',
        )
        github_stars = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[2]/div[2]',
        )
        github_forks = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[3]/div[2]',
        )
        github_contributors = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[4]/div[2]',
        )
        github_followers = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[5]/div[2]',
        )
        twitter_followers = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[6]/div[2]',
        )
        reddit_members = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[7]/div[2]',
        )

        list = [
            github_commits.text,
            github_stars.text,
            github_forks.text,
            github_contributors.text,
            github_followers.text,
            twitter_followers.text,
            reddit_members.text,
        ]

        list_new = [n.replace(",", "") for n in list]

        crypto_project_info.append(list_new)

    # Catch errarnous
    except NoSuchElementException:
        raise NoSuchElementException(f"{crypto.title()}")


NoSuchElementException: Message: cardano


In [20]:
github_commits = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]')
github_stars = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[2]/div[2]')
github_forks = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[3]/div[2]')
github_contributors = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[4]/div[2]')
github_followers = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[5]/div[2]')
twitter_followers = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[6]/div[2]')
reddit_members = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[7]/div[2]')

list = [
    github_commits.text, github_stars.text, github_forks.text, 
    github_contributors.text, github_followers.text, twitter_followers.text, reddit_members.text
    ]

list_new = [n.replace(',','') for n in list]
np.array(list_new)

In [21]:
//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]
//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]

array(['12706', '4229', '1405', '97', '506', '2569415'], dtype='<U7')

In [25]:
launch_chrome(f"https://coinmarketcap.com/currencies/cardano/project-info/")
driver.implicitly_wait(10)  # seconds

driver.maximize_window()

# Cookie element blocks
try:
    cookie = driver.find_element(By.XPATH, '//*[@id="cmc-cookie-policy-banner"]/div[2]')
    cookie.click()

    element = driver.find_element(
        By.XPATH,
        '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/div[4]',
    )
    WebDriverWait(driver, timeout=5)
    element.click()
    wait = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "selected"))
    )
except NoSuchElementException:
    pass


github_commits = driver.find_element(
    By.XPATH,
    '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]',
)
print(github_commits.text)
github_stars = driver.find_element(
    By.XPATH,
    '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[2]/div[2]',
)
github_forks = driver.find_element(
    By.XPATH,
    '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[3]/div[2]',
)
github_contributors = driver.find_element(
    By.XPATH,
    '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[4]/div[2]',
)
github_followers = driver.find_element(
    By.XPATH,
    '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[5]/div[2]',
)
twitter_followers = driver.find_element(
    By.XPATH,
    '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[6]/div[2]',
)
reddit_members = driver.find_element(
    By.XPATH,
    '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[7]/div[2]',
)

list = [
    github_commits.text,
    github_stars.text,
    github_forks.text,
    github_contributors.text,
    github_followers.text,
    twitter_followers.text,
    reddit_members.text,
]

list_new = [n.replace(",", "") for n in list]

crypto_project_info.append(list_new)

print(crypto_project_info)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/div[4]"}
  (Session info: chrome=106.0.5249.103)
Stacktrace:
0   chromedriver                        0x0000000105872598 chromedriver + 4404632
1   chromedriver                        0x00000001057f9fa3 chromedriver + 3911587
2   chromedriver                        0x00000001054a6e6a chromedriver + 425578
3   chromedriver                        0x00000001054dec91 chromedriver + 654481
4   chromedriver                        0x00000001054dee91 chromedriver + 654993
5   chromedriver                        0x0000000105516274 chromedriver + 881268
6   chromedriver                        0x00000001054fcdcd chromedriver + 777677
7   chromedriver                        0x0000000105513f79 chromedriver + 872313
8   chromedriver                        0x00000001054fcbd3 chromedriver + 777171
9   chromedriver                     

In [20]:
//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[2]/div[2]
//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[2]/div[2]

[['35628', '66489', '33099', '878', '3929', '--', '4634477'],
 ['13723', '39735', '15128', '834', '2181', '2910074', '1502623'],
 ['--', '--', '--', '--', '--', '299197', '--'],
 ['836', '358', '216', '17', '24', '--', '--'],
 ['1', '3750', '2045', '30', '506', '9571719', '876708'],
 ['12706', '4233', '1406', '97', '506', '2572297', '353502'],
 ['8', '95', '74', '4', '23', '9571719', '--']]

In [24]:
github_commits.text

'Organizations\nEMURGO Co., Ltd\nTokyo, Japan, Founded in 2017\nCardano Foundation\nZug, Switzerland, Founded in 2016\nInput Output Hong Kong\nHong Kong, Founded in 2015'

In [ ]:
top_crypto = [
    "bitcoin",
    "ethereum",
    "tether",
    "usd-coin",
    "bnb",
    "xrp",
    "binance-usd",
    "cardano",
    "solana",
    "dogecoin",
    "polkadot-new",
    "polygon",
    "tron",
    "HEX",
]

crypto_project_info = []

for crypto in top_crypto:
    try:
        launch_chrome(f"https://coinmarketcap.com/currencies/{crypto}/project-info/")
        driver.implicitly_wait(10)  # seconds

        driver.maximize_window()

        # Not all pages have an additional layer in dynamic component. If there is no dynamic component, the try block is not needed.
        try:
            # Cookie element blocks the selection of target element "Social Stats"
            cookie = driver.find_element(
                By.XPATH, '//*[@id="cmc-cookie-policy-banner"]/div[2]'
            )
            cookie.click()
            # Click on element "Social Stats" to access target data
            element = driver.find_element(
                By.XPATH,
                '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[1]/div[4]',
            )
            WebDriverWait(driver, timeout=5)
            element.click()
            wait = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "selected"))
            )
        except NoSuchElementException:
            pass

        github_commits = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[2]',
        )
        github_stars = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[2]/div[2]',
        )
        github_forks = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[3]/div[2]',
        )
        github_contributors = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[4]/div[2]',
        )
        github_followers = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[5]/div[2]',
        )
        twitter_followers = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[6]/div[2]',
        )
        reddit_members = driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[2]/div/div[2]/div[7]/div[2]',
        )

        list = [
            github_commits.text,
            github_stars.text,
            github_forks.text,
            github_contributors.text,
            github_followers.text,
            twitter_followers.text,
            reddit_members.text,
        ]

        list_new = [n.replace(",", "") for n in list]

        crypto_project_info.append(list_new)

    # Catch errarnous
    except NoSuchElementException:
        raise NoSuchElementException(f"{crypto.title()}")


In [43]:
launch_chrome('https://coinmarketcap.com/currencies/cardano/project-info/')

test = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div[1]/div[2]/div/div[3]/div[2]/div/div[1]')
for e in test:
    test = e.text.split('\n')
number_of_buttons = len(test)

if number_of_buttons == 2:
    2
elif number_of_buttons == 3:
    3
elif number_of_buttons == 4:
    4
else:
    1


Team
Token Distribution
Social Stats
['Team', 'Token Distribution', 'Social Stats']


3

In [41]:
mask_list

['Team\nToken Distribution\nSocial Stats']

In [ ]:
sc-1j943vo-0 gShtRn
sc-1j943vo-2 dNZpHx